In [1]:
!pip install pymorphy2

In [2]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [3]:
import requests

In [4]:
from bs4 import BeautifulSoup

In [5]:
import re

In [6]:
from nltk.tokenize import word_tokenize
from nltk import download
download('punkt')

[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
download ('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
from collections import Counter
import matplotlib.pyplot as plt

In [8]:
website_html = requests.get('https://opisanie-kartin.ru')

In [9]:
soup = BeautifulSoup(website_html.text)

In [10]:
all_li_to_artists = soup.find_all(class_=re.compile("menu\-item menu\-item\-type\-taxonomy menu\-item\-object\-category ((last-menu-item )|(first-menu-item ))?menu\-item\-\d{5}"))

In [11]:
artists_links = []
for Sub in all_li_to_artists:
  artists_links.append(Sub.find("a").get('href'))

In [ ]:
artists_links

In [12]:
paintings_list = []
for element in artists_links:
  painter_html = requests.get(element)
  soup_painter = BeautifulSoup(painter_html.text)
  painter_divs = soup_painter.find_all(class_='grid-entry-inner')

  for Sub in painter_divs:
    paintings_list.append(Sub.find("a").get('href'))

In [13]:
paintings_list

['https://opisanie-kartin.ru/ajvazovskij-ivan/prosshanie-pushkina-s-chernym-morem/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/more-koktebel/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/burya-na-more-nochyu/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/neapolitanskij-zaliv-v-lunnom-svete/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/petr-1-pri-krasnoj-gorke/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/chesmenskaya-bitva-nochyu/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/bosfor-v-lunnuyu-noch/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/neapolitanskij-zaliv-utrom/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/portret-loris-melikova/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/chesmenskaya-bitva/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/v-shtorm/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/vid-tiflisa-ot-seid-abaza/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/tonusshij-korabl/',
 'https://opisanie-kartin.ru/ajvazovskij-ivan/russko-tureckij-

In [14]:
def append_value(dictionary, key, value):
  if key in dictionary.keys():
    dictionary[key] + (value)
  else:
    dictionary[key] = value

In [15]:
genre_descriptions = {} 

In [16]:
for i in paintings_list:
  painting_html = requests.get(i)
  soup_painting = BeautifulSoup(painting_html.text)

  for p in soup_painting.find_all(class_='entry-content'):
    text = p.text
    genre = re.search("Стиль.+\.", text)
    
    text = text.lower()    
    if genre != None:
      genre = genre.group(0).replace(u'\xa0', u' ')
      
      file_list = word_tokenize(text)
      file_no_punct = [word for word in file_list if word[0].isalpha()]
      file_clean = [word for word in file_no_punct if word not in stop_words]
      
      words_lemmitized = []
      for token in file_clean:
        if len(token) > 1:
          lemma = morph.parse(token)[0].normal_form
          words_lemmitized.append(lemma)
      append_value(genre_descriptions, genre, words_lemmitized)

In [17]:
print(genre_descriptions.keys())

dict_keys(['Стиль: романтизм.', 'Стиль: неоклассицизм.', 'Стиль: северное Возрождение.', 'Стиль: классицизм.', 'Стиль: импрессионизм.', 'Стиль: реализм.', 'Стиль: маньеризм, позднее Возрождение.', 'Стиль: модерн.', 'Стиль: неоэкспрессионизм.', 'Стиль: постимпрессионизм.', 'Стиль: символизм.', 'Стиль: рококо.', 'Стиль: раннее Возрождение.', 'Стиль: аналитический кубизм.', 'Стиль: кубизм, экспрессионизм.', 'Стиль: фовизм.', 'Стиль: соцреализм.', 'Стиль: Неоклассицизм.', 'Стиль: высокое Возрождение.', 'Стиль: кубизм, футуризм.', 'Стиль: примитивизм.', 'Стиль: футуризм.', 'Стиль: японизм.', 'Стиль: барокко.', 'Стиль: романтизм, символизм.', 'Стиль: академизм.', 'Стиль: клуазонизм.', 'Стиль: сюрреализм.', 'Стиль: реализм, сюрреализм.', 'Стиль: экспрессионизм.', 'Стиль: примитивизм, сюрреализм.', 'Стиль: абстракционизм.', 'Стиль: кубизм, постимпрессионизм.', 'Стиль: импрессионизм, академизм.', 'Стиль: академизм, импрессионизм.', 'Стиль: абстрактный экспрессионизм.', 'Стиль: кубизм.', 'Стиль:

In [20]:
for key in list(genre_descriptions.keys()):
  frequency = Counter(genre_descriptions[key])
  most_common_words = frequency.most_common(50)
  values = [pair[1] for pair in most_common_words] 
  labels = [pair[0] for pair in most_common_words] 

  plt.figure(figsize=(20, 10))
  plt.title(key)
  plt.xticks(rotation=90)
  plt.bar(labels, values)
  plt.savefig(key + "png")
  plt.close('all')